##AKASH AND PRADYUMANNA KAGGLE CHALLENGE 1

In [165]:
# !pip install pycaret
# !pip install scorecardpy
# !pip install optbinning

In [166]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [167]:
from optbinning import ContinuousOptimalBinning

In [168]:
from pycaret.regression import *

In [169]:
le = preprocessing.LabelEncoder()

In [170]:
# from google.colab import drive
# drive.mount('/content/drive')

In [171]:
#loading test data
test_data = pd.read_csv("/content/drive/MyDrive/Kaggle 1/test_data.csv")

In [172]:
test_data

,index,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,0,Carroll,0,day_22,40,0.000000,0.000000,0.000000,0.000106,0.000000,...,58.2,4.7,713,47075,26589,88.5,56.9,14.8,116700,0.974052
1,1,Richland,3,day_118,17,0.000661,0.002567,0.058824,0.000494,0.000000,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
2,2,Portage,0,day_51,1210,0.000000,0.000000,0.000000,0.000013,0.000009,...,65.5,7.2,896,52552,25904,93.5,46.0,14.5,152000,0.909878
3,3,Auglaize,0,day_27,38,0.000000,0.000000,0.000000,0.000000,0.000000,...,67.0,3.4,758,54274,30438,95.7,59.4,9.0,143100,0.970007
4,4,Montgomery,0,day_8,4328,0.000005,0.000036,0.000462,0.000043,0.000033,...,61.8,8.0,830,43829,26292,92.0,44.8,17.9,112100,0.731997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,7326,Meigs,0,day_51,24,0.000000,0.000000,0.000000,0.000000,0.000473,...,52.8,10.4,571,37813,21714,91.7,51.9,22.5,87400,0.974262
7327,7327,Knox,3,day_109,128,0.000000,0.000000,0.000000,0.000033,0.000000,...,62.3,5.8,784,48533,24568,89.9,51.7,13.8,138900,0.965641
7328,7328,Columbiana,0,day_37,194,0.000000,0.000000,0.000000,0.000043,0.000000,...,58.0,7.5,656,44497,23781,91.3,52.0,15.4,104300,0.949782
7329,7329,Ashtabula,21,day_114,139,0.000000,0.000000,0.000000,0.000000,0.000000,...,54.7,7.4,695,40544,23117,90.0,47.9,19.8,106300,0.929083


In [173]:
#drop index columns 
test_data = test_data.drop(columns = 'index')

In [174]:
#adding a one column to test_data and 0 to train_data, we do this so that 
#we can perform same preprocessing on the whole data and later we split it 
#into train test as orignal. 
test_data['test_train'] = 1

In [175]:
test_data

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white,test_train
0,Carroll,0,day_22,40,0.000000,0.000000,0.000000,0.000106,0.000000,0.000000,...,4.7,713,47075,26589,88.5,56.9,14.8,116700,0.974052,1
1,Richland,3,day_118,17,0.000661,0.002567,0.058824,0.000494,0.000000,0.000000,...,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085,1
2,Portage,0,day_51,1210,0.000000,0.000000,0.000000,0.000013,0.000009,0.000036,...,7.2,896,52552,25904,93.5,46.0,14.5,152000,0.909878,1
3,Auglaize,0,day_27,38,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.4,758,54274,30438,95.7,59.4,9.0,143100,0.970007,1
4,Montgomery,0,day_8,4328,0.000005,0.000036,0.000462,0.000043,0.000033,0.000015,...,8.0,830,43829,26292,92.0,44.8,17.9,112100,0.731997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,Meigs,0,day_51,24,0.000000,0.000000,0.000000,0.000000,0.000473,0.000000,...,10.4,571,37813,21714,91.7,51.9,22.5,87400,0.974262,1
7327,Knox,3,day_109,128,0.000000,0.000000,0.000000,0.000033,0.000000,0.000000,...,5.8,784,48533,24568,89.9,51.7,13.8,138900,0.965641,1
7328,Columbiana,0,day_37,194,0.000000,0.000000,0.000000,0.000043,0.000000,0.000000,...,7.5,656,44497,23781,91.3,52.0,15.4,104300,0.949782,1
7329,Ashtabula,21,day_114,139,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.4,695,40544,23117,90.0,47.9,19.8,106300,0.929083,1


In [176]:
#reading train data
data = pd.read_csv("/content/drive/MyDrive/Kaggle 1/training_data.csv")

In [177]:
data

,county,cases,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,Richland,0,0,day_10,363,0.000000,0.000000,0.000000,0.000011,0.000151,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
1,Lawrence,119,0,day_105,256,0.000000,0.000000,0.000000,0.000000,0.000000,...,53.5,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027
2,Wayne,49,0,day_90,769,0.000076,0.000370,0.006502,0.000039,0.000000,...,64.0,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541
3,Fayette,7,0,day_85,36,0.000000,0.000000,0.000000,0.000000,0.000000,...,59.3,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054
4,Trumbull,0,0,day_7,554,0.000000,0.000000,0.000000,0.000046,0.000000,...,56.4,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,Summit,105,0,day_81,6121,0.000094,0.000491,0.008169,0.000041,0.000011,...,64.2,6.4,859,50765,28345,93.2,47.3,13.6,137000,0.791435
3137,Fayette,0,0,day_55,28,0.000000,0.000000,0.000000,0.000000,0.000000,...,59.3,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054
3138,Clark,147,0,day_106,929,0.000013,0.000105,0.001076,0.000009,0.000012,...,60.6,7.7,736,43625,25300,92.7,48.1,16.6,107300,0.865754
3139,Logan,0,0,day_1,144,0.000000,0.000000,0.000000,0.000000,0.000000,...,62.5,5.8,766,49783,28346,90.7,55.2,13.7,127200,0.949363


In [178]:
#reading labels
train_y = data['cases']

In [179]:
#drop y variable from the training data
data = data.drop(columns= 'cases')

In [180]:
#explained above
data['test_train'] = 0

In [181]:
#merge the two dataframes
data = pd.concat([data,test_data])

In [182]:
data

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white,test_train
0,Richland,0,day_10,363,0.000000,0.00000,0.000000,0.000011,0.000151,0.000053,...,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085,0
1,Lawrence,0,day_105,256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027,0
2,Wayne,0,day_90,769,0.000076,0.00037,0.006502,0.000039,0.000000,0.000000,...,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541,0
3,Fayette,0,day_85,36,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054,0
4,Trumbull,0,day_7,554,0.000000,0.00000,0.000000,0.000046,0.000000,0.000000,...,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,Meigs,0,day_51,24,0.000000,0.00000,0.000000,0.000000,0.000473,0.000000,...,10.4,571,37813,21714,91.7,51.9,22.5,87400,0.974262,1
7327,Knox,3,day_109,128,0.000000,0.00000,0.000000,0.000033,0.000000,0.000000,...,5.8,784,48533,24568,89.9,51.7,13.8,138900,0.965641,1
7328,Columbiana,0,day_37,194,0.000000,0.00000,0.000000,0.000043,0.000000,0.000000,...,7.5,656,44497,23781,91.3,52.0,15.4,104300,0.949782,1
7329,Ashtabula,21,day_114,139,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,7.4,695,40544,23117,90.0,47.9,19.8,106300,0.929083,1


In [183]:
#convert string features into numeric features
#using label encder
data['county'] = le.fit_transform(data['county'])
data['date_index_converted'] = le.fit_transform(data['date_index_converted'])

In [184]:
data

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white,test_train
0,69,0,1,363,0.000000,0.00000,0.000000,0.000011,0.000151,0.000053,...,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085,0
1,43,0,7,256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027,0
2,84,0,109,769,0.000076,0.00037,0.006502,0.000039,0.000000,0.000000,...,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541,0
3,23,0,103,36,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054,0
4,77,0,86,554,0.000000,0.00000,0.000000,0.000046,0.000000,0.000000,...,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,52,0,66,24,0.000000,0.00000,0.000000,0.000000,0.000473,0.000000,...,10.4,571,37813,21714,91.7,51.9,22.5,87400,0.974262,1
7327,41,3,11,128,0.000000,0.00000,0.000000,0.000033,0.000000,0.000000,...,5.8,784,48533,24568,89.9,51.7,13.8,138900,0.965641,1
7328,14,0,50,194,0.000000,0.00000,0.000000,0.000043,0.000000,0.000000,...,7.5,656,44497,23781,91.3,52.0,15.4,104300,0.949782,1
7329,3,21,17,139,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,7.4,695,40544,23117,90.0,47.9,19.8,106300,0.929083,1


In [185]:
#list of features that need to be normalised
list_ = ['percent_married',
 'labor_force_rate',
 'median_worker_earnings',
 'median_housing_cost',
 'percent_white',
 'percent_25_34',
 'poverty_rate',
 'percent_highschool',
 'percent_insured',
 'median_household_earnings',
 'median_property_value']
for column in list_:
  data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())    


In [186]:
data

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white,test_train
0,69,0,1,363,0.000000,0.00000,0.000000,0.000011,0.000151,0.000053,...,7.5,0.182609,0.137820,0.296714,0.831978,0.484429,0.418327,0.093203,0.655486,0
1,43,0,7,256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.5,0.163285,0.154985,0.306762,0.878049,0.519031,0.537849,0.081874,0.891626,0
2,84,0,109,769,0.000076,0.00037,0.006502,0.000039,0.000000,0.000000,...,4.0,0.266667,0.284266,0.412198,0.742547,0.719723,0.314741,0.280639,0.882049,0
3,23,0,103,36,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.3,0.237681,0.114164,0.385404,0.853659,0.605536,0.501992,0.119979,0.853233,0
4,77,0,86,554,0.000000,0.00000,0.000000,0.000046,0.000000,0.000000,...,5.9,0.169082,0.158411,0.383461,0.864499,0.512111,0.482072,0.082389,0.703952,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,52,0,66,24,0.000000,0.00000,0.000000,0.000000,0.000473,0.000000,...,10.4,0.082126,0.067851,0.246609,0.864499,0.608997,0.693227,0.009269,0.947226,1
7327,41,3,11,128,0.000000,0.00000,0.000000,0.000033,0.000000,0.000000,...,5.8,0.287923,0.252415,0.342198,0.815718,0.602076,0.346614,0.274459,0.923539,1
7328,14,0,50,194,0.000000,0.00000,0.000000,0.000043,0.000000,0.000000,...,7.5,0.164251,0.182928,0.315839,0.853659,0.612457,0.410359,0.096292,0.879963,1
7329,3,21,17,139,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,7.4,0.201932,0.114870,0.293599,0.818428,0.470588,0.585657,0.106591,0.823088,1


In [187]:
#awareness variables
columns_pca = '''core_cosine, core_cosine_normalized, core_intersection, core_intersection_nor
malized, core_jaccard, core_jaccard_normalized, domestic_cosine, domestic_cos
ine_normalized, domestic_intersection, domestic_intersection_normalized, dome
stic_jaccard, domestic_jaccard_normalized, economy_cosine, economy_cosine_nor
malized, economy_intersection, economy_intersection_normalized, economy_jacca
rd, economy_jaccard_normalized, education_cosine, education_cosine_normalized
, education_intersection, education_intersection_normalized, education_jaccar
d, education_jaccard_normalized, entertainment_cosine, entertainment_cosine_n
ormalized, entertainment_intersection, entertainment_intersection_normalized,
entertainment_jaccard, entertainment_jaccard_normalized, foreign_cosine, fore
ign_cosine_normalized, foreign_intersection, foreign_intersection_normalized,
foreign_jaccard, foreign_jaccard_normalized, gender_cosine, gender_cosine_nor
malized, gender_intersection, gender_intersection_normalized, gender_jaccard,
gender_jaccard_normalized, health_cosine, health_cosine_normalized, health_in
tersection, health_intersection_normalized, health_jaccard, health_jaccard_no
rmalized, health_technology_cosine, health_technology_cosine_normalized, heal
th_technology_intersection, health_technology_intersection_normalized, health
_technology_jaccard, health_technology_jaccard_normalized, ideology_cosine, i
deology_cosine_normalized, ideology_intersection, ideology_intersection_norma
lized, ideology_jaccard, ideology_jaccard_normalized, illness_cosine, illness
_cosine_normalized, illness_intersection, illness_intersection_normalized, il
lness_jaccard, illness_jaccard_normalized, labor_force_rate, median_household
_earnings, median_housing_cost, median_property_value, median_worker_earnings,
nationalistic_cosine, nationalistic_cosine_normalized, nationalistic_intersec
tion, nationalistic_intersection_normalized, nationalistic_jaccard, nationali
stic_jaccard_normalized, percent_25_34, percent_highschool, percent_insured, p
ercent_married, percent_white, politics_cosine, politics_cosine_normalized, p
olitics_democratic_hate_cosine, politics_democratic_hate_cosine_normalized, p
olitics_democratic_hate_intersection, politics_democratic_hate_intersection_n
ormalized, politics_democratic_hate_jaccard, politics_democratic_hate_jaccard
_normalized, politics_democratic_love_cosine, politics_democratic_love_cosine
_normalized, politics_democratic_love_intersection, politics_democratic_love_
intersection_normalized, politics_democratic_love_jaccard, politics_democrati
c_love_jaccard_normalized, politics_intersection, politics_intersection_norma
lized, politics_jaccard, politics_jaccard_normalized, politics_republican_hat
e_cosine, politics_republican_hate_cosine_normalized, politics_republican_hat
e_intersection, politics_republican_hate_intersection_normalized, politics_re
publican_hate_jaccard, politics_republican_hate_jaccard_normalized, politics_
republican_love_cosine, politics_republican_love_cosine_normalized, politics_
republican_love_intersection, politics_republican_love_intersection_normalize
d, politics_republican_love_jaccard, politics_republican_love_jaccard_normali
zed, poverty_rate, race_cosine, race_cosine_normalized, race_intersection, ra
ce_intersection_normalized, race_jaccard, race_jaccard_normalized, religion_c
osine, religion_cosine_normalized, religion_intersection, religion_intersecti
on_normalized, religion_jaccard, religion_jaccard_normalized, social_cosine,
social_cosine_normalized, social_intersection, social_intersection_normalized
, social_jaccard, social_jaccard_normalized, sports_cosine, sports_cosine_nor
malized, sports_intersection, sports_intersection_normalized, sports_jaccard,
sports_jaccard_normalized'''

In [188]:
#preprocessing to read the variables
columns_pca = columns_pca.replace(" ","")
columns_pca = columns_pca.split(",")
columns_pca = list(map(lambda x: x.replace("\n",""),columns_pca))

In [189]:
#in the next few cells we will divide the columns into normalised and non normalised 
data_pca_to_be_reduced_updated_columns_normalized = []
#temp_list = []
for val in columns_pca:
  if val.split("_")[-1] == 'normalized':
     data_pca_to_be_reduced_updated_columns_normalized.append(val)
     #temp_list.append(str(val.split("_")[0]+ "_" +val.split("_")[1]))

In [190]:
#in the next few cells we will divide the columns into normalised and non normalised 

data_pca_to_be_reduced_updated_columns_not_normalized = [val.replace("_normalized","") for val in columns_pca ]

In [191]:
#in the next few cells we will divide the columns into normalised and non normalised 

data_pca_to_be_reduced_updated_columns_not_normalized = set(data_pca_to_be_reduced_updated_columns_not_normalized)

In [192]:
len(data_pca_to_be_reduced_updated_columns_not_normalized)

74

In [193]:
len(data_pca_to_be_reduced_updated_columns_normalized)

63

In [194]:
#in the next few cells we will divide the columns into normalised and non normalised 

extra_columns = [val for val in data_pca_to_be_reduced_updated_columns_not_normalized if val+ "_normalized" not in data_pca_to_be_reduced_updated_columns_normalized]

In [195]:
extra_columns

['labor_force_rate',
 'median_household_earnings',
 'percent_insured',
 'percent_25_34',
 'median_property_value',
 'percent_married',
 'percent_highschool',
 'median_housing_cost',
 'percent_white',
 'poverty_rate',
 'median_worker_earnings']

In [196]:
#in the next few cells we will divide the columns into normalised and non normalised 

data_pca_to_be_reduced = data[data_pca_to_be_reduced_updated_columns_normalized]

In [197]:
data_pca_to_be_reduced

,core_cosine_normalized,core_intersection_normalized,core_jaccard_normalized,domestic_cosine_normalized,domestic_intersection_normalized,domestic_jaccard_normalized,economy_cosine_normalized,economy_intersection_normalized,economy_jaccard_normalized,education_cosine_normalized,...,race_jaccard_normalized,religion_cosine_normalized,religion_intersection_normalized,religion_jaccard_normalized,social_cosine_normalized,social_intersection_normalized,social_jaccard_normalized,sports_cosine_normalized,sports_intersection_normalized,sports_jaccard_normalized
0,0.000000,0.00000,0.000000,0.007971,0.005510,0.005499,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000831,0.002755,0.002632,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006819,0.007668,0.007589
2,0.020211,0.03251,0.032428,0.013643,0.007802,0.007849,0.001537,0.002663,0.002623,0.005202,...,0.0,0.009112,0.005852,0.005979,0.006649,0.009103,0.009071,0.002539,0.002553,0.002544
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.003799,0.004513,0.004465,0.002236,0.007392,0.006867,0.000000,...,0.0,0.000000,0.000000,0.000000,0.008167,0.010830,0.010763,0.039299,0.021260,0.021430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7327,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.015997,0.015997,0.015987,0.000000,...,0.0,0.086346,0.070312,0.070791,0.007812,0.007812,0.007802,0.000000,0.000000,0.000000
7328,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.054450,0.115979,0.109451,0.004209,0.010309,0.010129,0.010061,0.010118,0.010071
7329,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [198]:
#implenting PCA on the normalised data
pca = PCA(n_components = 0.95)

In [199]:
pca.fit(data_pca_to_be_reduced)

PCA(n_components=0.95)

In [200]:
#implenting PCA on the normalised data

reduced_data = pca.transform(data_pca_to_be_reduced)

In [201]:
len(reduced_data[0])

18

In [202]:
#adding reduced features to main data
columns = [ "PCA_feature"+str(i) for i in range(len(reduced_data[0]))]
reduced_data_df = pd.DataFrame(reduced_data, columns = columns)

In [203]:
reduced_data_df

,PCA_feature0,PCA_feature1,PCA_feature2,PCA_feature3,PCA_feature4,PCA_feature5,PCA_feature6,PCA_feature7,PCA_feature8,PCA_feature9,PCA_feature10,PCA_feature11,PCA_feature12,PCA_feature13,PCA_feature14,PCA_feature15,PCA_feature16,PCA_feature17
0,-0.031262,-0.042357,-0.008507,-0.005472,-0.024004,0.001003,0.016813,0.005622,-0.016566,0.017463,-0.008467,-0.002022,0.001002,0.024597,0.015042,-0.002483,0.003565,-0.003395
1,-0.026458,-0.026409,0.012513,-0.005044,0.096876,-0.025615,0.010982,0.001025,-0.024782,-0.015195,-0.002721,-0.003287,-0.007146,0.000915,-0.010960,-0.006906,0.004683,-0.001406
2,0.045648,-0.026148,-0.002622,-0.003800,-0.030263,-0.004662,-0.008218,-0.008812,0.001592,-0.006746,-0.002695,0.000206,0.006420,-0.001120,-0.001099,-0.003214,0.000799,-0.003971
3,0.000623,-0.021053,0.099636,0.002303,0.537993,-0.131441,-0.032429,0.012093,-0.067105,-0.004129,0.004736,-0.005778,-0.014199,-0.010755,0.003713,-0.000796,0.006315,-0.000183
4,-0.031111,0.008903,-0.001327,-0.006071,0.076526,-0.023128,-0.015367,-0.003522,-0.018014,-0.007951,0.003464,0.001046,0.010110,-0.002329,0.000490,-0.002846,0.001578,-0.003146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10467,-0.030728,-0.042678,-0.008570,-0.005391,-0.023012,-0.001054,0.018943,0.000724,-0.010075,-0.005626,-0.004809,-0.003731,-0.006601,0.094994,0.091429,0.008914,0.002730,-0.003692
10468,-0.026202,-0.041594,-0.012572,-0.002374,-0.013017,-0.012636,0.023840,-0.044533,0.107121,0.021590,0.023998,0.002889,0.004099,-0.005919,-0.001108,-0.006001,0.001611,-0.002536
10469,-0.025427,-0.025339,-0.010332,-0.003333,0.023645,-0.022506,0.067805,-0.049761,0.124819,0.015354,-0.000349,0.004078,0.005775,-0.002955,-0.015745,-0.013107,0.002132,-0.002225
10470,-0.031941,-0.042862,-0.008611,-0.005829,-0.025136,-0.004420,-0.010556,-0.005565,-0.007291,-0.008987,-0.005501,-0.002766,-0.006561,-0.000740,-0.000197,-0.002790,0.003931,-0.001940


In [204]:
#removing the non normalised counterparts of the normalised features
columns_to_be_removed = data_pca_to_be_reduced_updated_columns_normalized + list(data_pca_to_be_reduced_updated_columns_not_normalized)

In [205]:
columns_to_be_removed = list(filter(lambda i: i not in extra_columns, columns_to_be_removed))

In [206]:
len(columns_to_be_removed)

126

In [207]:
data = data.drop(columns=columns_to_be_removed)

In [208]:
data = data.reset_index()

In [209]:
data

,index,county,deaths,date_index_converted,county_data_length,total_pop,percent_25_34,percent_highschool,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white,test_train
0,0,69,0,1,363,122312,0.402062,0.745455,0.575221,7.5,0.182609,0.137820,0.296714,0.831978,0.484429,0.418327,0.093203,0.655486,0
1,1,43,0,7,256,61827,0.371134,0.719481,0.516224,6.5,0.163285,0.154985,0.306762,0.878049,0.519031,0.537849,0.081874,0.891626,0
2,2,84,0,109,769,115371,0.350515,0.716883,0.825959,4.0,0.266667,0.284266,0.412198,0.742547,0.719723,0.314741,0.280639,0.882049,0
3,3,23,0,103,36,28769,0.360825,0.750649,0.687316,6.3,0.237681,0.114164,0.385404,0.853659,0.605536,0.501992,0.119979,0.853233,0
4,4,77,0,86,554,206373,0.298969,0.800000,0.601770,5.9,0.169082,0.158411,0.383461,0.864499,0.512111,0.482072,0.082389,0.703952,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10467,7326,52,0,66,24,23473,0.298969,0.680519,0.495575,10.4,0.082126,0.067851,0.246609,0.864499,0.608997,0.693227,0.009269,0.947226,1
10468,7327,41,3,11,128,61004,0.268041,0.849351,0.775811,5.8,0.287923,0.252415,0.342198,0.815718,0.602076,0.346614,0.274459,0.923539,1
10469,7328,14,0,50,194,105987,0.288660,0.761039,0.648968,7.5,0.164251,0.182928,0.315839,0.853659,0.612457,0.410359,0.096292,0.879963,1
10470,7329,3,21,17,139,99777,0.288660,0.714286,0.551622,7.4,0.201932,0.114870,0.293599,0.818428,0.470588,0.585657,0.106591,0.823088,1


In [210]:
data = data.drop(columns = 'index')

In [211]:
data.shape

(10472, 18)

In [212]:
#concatnating reduced features we got from PCA and normal features
data = pd.concat([data,reduced_data_df],axis = 1)

In [213]:
data['test_train']

0        0
1        0
2        0
3        0
4        0
        ..
10467    1
10468    1
10469    1
10470    1
10471    1
Name: test_train, Length: 10472, dtype: int64

In [214]:
#spliting data into train test split as given orignally
train_data = data[data['test_train'] == 0]
test_data = data[data['test_train'] == 1]

In [215]:
train_y.values

array([  0, 119,  49, ..., 147,   0,   0])

In [216]:
#function to perform binning of numeric variables
def binning(column,train_data,train_y):
  train_x = train_data[column].values
  y = train_y.values
  optb = ContinuousOptimalBinning(name=column, dtype="numerical")
  optb.fit(train_x, y)
  test_data[column] = optb.transform(test_data[column])
  train_data[column] = optb.transform(train_data[column])
  le.fit(train_data[column])
  train_data[column] = le.transform(train_data[column])
  test_data[column] = le.transform(test_data[column])

In [217]:
columns_to_bin = list(data.columns)

In [218]:
#removing columns we cannot bin
columns_to_bin = [val for val in columns_to_bin if val not in ['cases','test_train']]

In [219]:
#implementing binning
for columns in columns_to_bin:
  splits = binning(columns,train_data,train_y)

In [220]:
#adding one to all bins so that there are no bins labeled 0
columns = train_data.columns.tolist()
columns.remove('county')
columns.remove('date_index_converted')
def fun(val):
  return val+1
for cols in columns:
  train_data[cols] = train_data[cols].apply(fun)
  test_data[cols] = test_data[cols].apply(fun)

In [221]:
#droping internal use variables
test_data = test_data.drop(columns='test_train')
train_data = train_data.drop(columns='test_train')

In [222]:
test_data

,county,deaths,date_index_converted,county_data_length,total_pop,percent_25_34,percent_highschool,labor_force_rate,unemployment_rate,median_housing_cost,...,PCA_feature8,PCA_feature9,PCA_feature10,PCA_feature11,PCA_feature12,PCA_feature13,PCA_feature14,PCA_feature15,PCA_feature16,PCA_feature17
3141,5,1,0,2,2,1,9,3,2,3,...,1,2,2,5,7,2,4,2,6,4
3142,3,2,5,1,8,4,4,2,5,5,...,6,2,5,5,7,4,5,4,7,7
3143,3,1,0,6,8,2,6,4,5,6,...,7,4,6,3,6,4,5,7,1,1
3144,5,1,0,2,6,2,5,4,3,1,...,2,2,3,2,2,2,2,2,4,3
3145,3,1,2,7,10,5,8,3,6,6,...,7,5,5,4,6,5,5,7,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10467,3,1,0,1,2,2,2,1,7,5,...,4,5,1,3,1,5,5,7,3,4
10468,2,2,5,3,7,2,7,3,4,6,...,7,4,8,4,6,4,6,5,2,2
10469,5,1,0,3,8,2,9,3,5,5,...,7,4,6,4,6,4,6,6,1,3
10470,5,3,5,3,7,2,3,2,5,3,...,2,2,3,2,2,2,2,2,4,3


In [325]:
train_data 

,county,deaths,date_index_converted,county_data_length,total_pop,percent_25_34,percent_highschool,labor_force_rate,unemployment_rate,median_housing_cost,...,PCA_feature8,PCA_feature9,PCA_feature10,PCA_feature11,PCA_feature12,PCA_feature13,PCA_feature14,PCA_feature15,PCA_feature16,PCA_feature17
0,3,1,5,4,8,4,4,2,5,5,...,5,4,5,4,6,5,5,2,4,1
1,2,1,5,4,7,4,3,2,4,5,...,6,2,6,3,3,5,6,6,5,7
2,3,1,4,5,8,4,3,4,2,4,...,7,3,6,4,6,2,6,3,6,4
3,5,1,3,2,3,4,4,3,4,2,...,6,5,7,3,5,4,5,7,5,7
4,3,1,0,5,8,2,10,3,4,5,...,6,2,6,4,6,4,5,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,3,1,2,8,10,4,7,4,4,6,...,7,2,6,1,6,5,6,6,6,5
3137,5,1,0,2,3,4,4,3,4,2,...,6,2,5,3,3,2,3,6,3,3
3138,5,1,5,6,8,3,4,3,6,2,...,7,5,7,4,4,5,5,3,3,2
3139,2,1,5,3,6,2,8,3,4,4,...,3,2,5,4,6,3,3,5,1,2


In [302]:
from sklearn.feature_selection import RFE

In [324]:
#cheaking for multi-colinearity
for i in range(len(train_data)):
  print(train_data.columns[i],variance_inflation_factor(train_data.values, i))
# county 7.169987595194913
# deaths 10.137726646118827
# date_index_converted 2.621397599673016
# county_data_length 23.421271753566575 ---
# total_pop 27.252239015694983 ----
# percent_25_34 13.665692073037667
# percent_highschool 12.27807580802806
# labor_force_rate 26.714219240721352 ----
# unemployment_rate 14.220852099093351
# median_housing_cost 10.834269109115034
# median_household_earnings 43.49534864195028 ---
# median_worker_earnings 36.60894645700685 ---
# percent_insured 7.634968701655601
# percent_married 21.693202348250885 ----
# poverty_rate 11.033872287861351
#best_columns = ['county','deaths','date_index_converted','percent_highschool','unemployment_rate','median_housing_cost','percent_insured','poverty_rate']

county 7.169987595194913
deaths 10.137726646118827
date_index_converted 2.621397599673016
county_data_length 23.421271753566575
total_pop 27.252239015694983
percent_25_34 13.665692073037667
percent_highschool 12.27807580802806
labor_force_rate 26.714219240721352
unemployment_rate 14.220852099093351
median_housing_cost 10.834269109115034
median_household_earnings 43.49534864195028
median_worker_earnings 36.60894645700685
percent_insured 7.634968701655601
percent_married 21.693202348250885
poverty_rate 11.033872287861351
median_property_value 31.727994731513025
percent_white 20.180896125157197
PCA_feature0 10.218352814550876
PCA_feature1 17.269000787411265
PCA_feature2 6.231198243800076
PCA_feature3 15.417939834127646
PCA_feature4 13.015036716474802
PCA_feature5 7.0923066292181876
PCA_feature6 9.412644363083489
PCA_feature7 11.99009016973355
PCA_feature8 14.202033092544566
PCA_feature9 7.006534699925476
PCA_feature10 10.71412744822433
PCA_feature11 25.552706645752604
PCA_feature12 18.160

IndexError: ignored

In [240]:
#loading model
reg = ExtraTreesRegressor(n_estimators=100, random_state=0)

In [304]:
#identifying important features using Feature elimination
rfe = RFE(reg, n_features_to_select=11, step=1)
fit = rfe.fit(train_data, train_y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 11
Selected Features: [ True  True  True  True  True  True False False False False False  True
  True  True False False  True False  True False False False False False
 False False False False False False False False False False False]
Feature Ranking: [ 1  1  1  1  1  1  5  7 12 10  8  1  1  1  2  3  1 20  1 18 17 22 13 11
 15 23  6  9 16 24  4 25 14 19 21]


In [305]:
#selecting best columns using RFE
best_columns = []
for x,y in zip(fit.feature_names_in_,fit.ranking_):
  if y == 1:
    best_columns.append(x)
    print(x)
  #print(x,y)

county
deaths
date_index_converted
county_data_length
total_pop
percent_25_34
median_worker_earnings
percent_insured
percent_married
percent_white
PCA_feature1


In [264]:
print(best_columns)

['county', 'deaths', 'date_index_converted', 'county_data_length', 'total_pop', 'percent_25_34', 'percent_highschool', 'median_worker_earnings', 'percent_insured', 'percent_married', 'poverty_rate', 'median_property_value', 'percent_white', 'PCA_feature1', 'PCA_feature13']


In [306]:
# best_columns = ['county', 'deaths', 'date_index_converted', 'county_data_length',
#        'total_pop', 'percent_25_34','median_worker_earnings',
#        'percent_insured', 'percent_married','percent_white','PCA_feature1']

# best_columns = ['county', 'deaths','county_data_length','total_pop','median_worker_earnings','percent_25_34','percent_insured','median_property_value',
#                 'percent_white','percent_married']

train_data_specific = train_data[best_columns]
test_data_specific = test_data[best_columns]

In [276]:
train_data_specific

,county,deaths,date_index_converted,county_data_length,total_pop,percent_25_34,median_worker_earnings,percent_insured,percent_married,percent_white,PCA_feature1
0,3,1,5,4,8,4,3,1,4,6,2
1,2,1,5,4,7,4,3,6,4,4,4
2,3,1,4,5,8,4,7,1,2,4,4
3,5,1,3,2,3,4,6,6,3,5,4
4,3,1,0,5,8,2,6,6,4,6,4
...,...,...,...,...,...,...,...,...,...,...,...
3136,3,1,2,8,10,4,8,5,5,7,4
3137,5,1,0,2,3,4,6,6,3,5,5
3138,5,1,5,6,8,3,5,9,4,6,4
3139,2,1,5,3,6,2,8,1,2,4,5


In [319]:
reg = ExtraTreesRegressor(n_estimators=100, random_state=0)

In [320]:
#training ExtraTreesRegressor
ExtraTreesRegressor_results = cross_validate(estimator=reg,
                               X= train_data_specific,
                               y= train_y,
                               scoring="r2",
                               return_train_score=True)
ExtraTreesRegressor_results

{'fit_time': array([0.35197091, 0.34820437, 0.34560108, 0.35522294, 0.35323405]),
 'score_time': array([0.01768851, 0.0169208 , 0.01698184, 0.01703024, 0.02435231]),
 'test_score': array([0.7811315 , 0.91496318, 0.9529882 , 0.96505071, 0.93644351]),
 'train_score': array([0.99587371, 0.99037725, 0.99199597, 0.9941847 , 0.9952943 ])}

In [321]:
pd.DataFrame.from_dict(ExtraTreesRegressor_results)

,fit_time,score_time,test_score,train_score
0,0.351971,0.017689,0.781131,0.995874
1,0.348204,0.016921,0.914963,0.990377
2,0.345601,0.016982,0.952988,0.991996
3,0.355223,0.017030,0.965051,0.994185
4,0.353234,0.024352,0.936444,0.995294


In [297]:
#fitting ExtraTreesRegressor on full data
reg.fit(train_data_specific, train_y)

ExtraTreesRegressor(random_state=0)

In [299]:
#predicting on test data
pred = reg.predict(test_data_specific)
#test_data = test_data.reset_index()
df = pd.DataFrame.from_dict({
    "Index":test_data.index,
    "Cases":pred
})

In [301]:
df.to_csv("Submission_bhagwan_bachale_3.csv",index = False)

In [261]:
df.to_csv("Submission_bhagwan_bachale_2.csv",index = False)

In [ ]:
test_data_specific

In [326]:
#setting up pycaret
reg = setup(data=train_data, target=train_y, session_id=42)

,Description,Value
0,Session id,42
1,Target,cases
2,Target type,Regression
3,Original data shape,"(3141, 36)"
4,Transformed data shape,"(3141, 36)"
5,Transformed train set shape,"(2198, 36)"
6,Transformed test set shape,"(943, 36)"
7,Numeric features,35
8,Preprocess,True
9,Imputation type,simple


In [327]:
#using pycaret to find best model
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,59.7130,152180.9815,334.8554,0.7841,0.9575,0.8889,1.0390
gbr,Gradient Boosting Regressor,82.0978,147519.1207,330.5989,0.7622,2.1000,1.3105,0.8380
xgboost,Extreme Gradient Boosting,63.4454,200057.1663,376.9769,0.7570,1.2500,0.7315,0.5200
rf,Random Forest Regressor,67.0368,183034.6611,366.3997,0.7236,0.9959,0.8233,1.0500
lightgbm,Light Gradient Boosting Machine,94.8640,181021.5785,381.8553,0.6964,2.1161,1.7893,0.2930
ada,AdaBoost Regressor,167.6588,201553.0141,414.7210,0.5971,3.7093,4.1809,0.3480
dt,Decision Tree Regressor,78.1604,287443.0954,464.1123,0.5371,1.0196,0.8126,0.0680
huber,Huber Regressor,128.2185,565531.9701,667.5433,0.2840,2.0743,1.6094,0.2970
par,Passive Aggressive Regressor,264.2514,563779.7668,666.8415,0.2831,3.9350,6.0918,0.1000
knn,K Neighbors Regressor,119.9334,360741.4329,572.3608,0.2380,1.6485,2.2767,0.1090


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [329]:
#running cross validation on best model
model = create_model(best_model)

# evaluate the model performance on the test set
predict_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,92.1618,286978.2460,535.7035,0.5269,1.0641,0.6160
1,45.1420,24855.9465,157.6577,0.8136,0.9458,2.3678
2,48.1185,48573.8552,220.3948,0.8758,0.9548,0.5911
3,27.4038,7413.2702,86.1003,0.9111,0.8655,0.7227
4,84.3881,272074.6632,521.6078,0.8644,0.9524,0.5683
5,65.7063,230653.1505,480.2636,0.7868,1.0856,0.5462
6,94.0573,502819.3500,709.0976,0.3792,1.0629,1.0387
7,59.4895,51424.3153,226.7693,0.9417,0.9448,0.8789
8,28.7128,15924.4664,126.1922,0.8380,0.8404,0.7961


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,68.3156,163654.5785,404.5424,0.8167,0.8032,0.7344


,county,deaths,date_index_converted,county_data_length,total_pop,percent_25_34,percent_highschool,labor_force_rate,unemployment_rate,median_housing_cost,...,PCA_feature10,PCA_feature11,PCA_feature12,PCA_feature13,PCA_feature14,PCA_feature15,PCA_feature16,PCA_feature17,cases,prediction_label
2894,0,1,5,3,1,1,4,2,1,5,...,8,3,6,5,6,5,7,8,0,9.24
2969,1,1,2,1,5,4,1,4,3,3,...,3,2,2,2,2,2,4,3,0,0.00
1064,5,1,4,4,7,2,3,2,5,3,...,6,3,6,3,6,7,6,8,140,72.38
1231,4,1,0,2,3,4,1,1,4,5,...,3,2,2,2,2,2,4,3,0,0.00
2988,0,1,1,3,5,2,2,2,5,3,...,4,4,6,2,1,2,3,2,0,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,5,1,1,4,5,2,8,3,4,5,...,5,3,5,4,1,5,5,7,0,1.47
748,4,1,1,3,5,2,7,4,4,6,...,5,4,6,2,5,5,7,8,0,1.19
1190,3,1,3,3,6,5,4,2,4,6,...,4,4,6,4,6,3,5,2,21,14.49
662,2,1,0,6,9,4,6,4,2,6,...,8,3,1,5,5,7,6,8,0,0.00


##XGBRegressor

In [311]:
from xgboost import XGBRegressor

In [317]:
#loading XGBoost
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [318]:
#CV on XGBoost
XGBRegressor_results = cross_validate(estimator=model,
                               X= train_data_specific,
                               y= train_y,
                               scoring="r2",
                               return_train_score=True)
XGBRegressor_results

{'fit_time': array([4.07147741, 2.62808728, 2.57390547, 6.9312408 , 2.50536346]),
 'score_time': array([0.02318358, 0.02559304, 0.0231576 , 0.02293062, 0.02301407]),
 'test_score': array([0.74499292, 0.93160102, 0.94850421, 0.93832791, 0.93980731]),
 'train_score': array([0.99577234, 0.99026215, 0.99187893, 0.99408564, 0.99523792])}

In [322]:
pd.DataFrame.from_dict(XGBRegressor_results)

,fit_time,score_time,test_score,train_score
0,4.071477,0.023184,0.744993,0.995772
1,2.628087,0.025593,0.931601,0.990262
2,2.573905,0.023158,0.948504,0.991879
3,6.931241,0.022931,0.938328,0.994086
4,2.505363,0.023014,0.939807,0.995238


In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)